# IY master
医薬品情報を日付に対応したマスターとして作成する。
薬価は改定月の翌月2日から有効なものとなる。

In [1]:
import pandas as pd
import json
import pathlib
import datetime
import numpy as np

ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")
ts

'20231122_150053_929635'

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 250)

マスター読み込み関数  
マスターにはカラム名が無いので、c+番号でカラム名をつける

In [3]:
# f = "/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tensuhyo/kihonmasta/kihonmasta_04.files/y_20220304.csv"
# df = pd.read_csv(f,encoding='cp932', header=None, dtype={29:str})
# df.head().T

In [4]:
def get_file_df(filepath):
    # c29 is date of update
    df_i = pd.read_csv(filepath,encoding='cp932', header=None, dtype={29:str, 30:str, 33:str})
    df_i.columns = ["c"+ str(i) for i in range(len(df_i.columns))]
    df_i = df_i
    df_i["filepath"] = f
    return df_i

In [5]:
# f_pk = "./masterfile_20230426_150151_224504.pkl"
# f_pk = "./masterfile_20230427_095353_581518.pkl"
# f_pk = "./masterfile_20230603_100705_814763.pkl"
f_pk = "./masterfile_20231122_100722_065215.pkl"
df_files = pd.read_pickle(f_pk)

df_files.head()

,filepath,filename,parent,gparent,bearfile,timeinfo,prefix,ymdinfo,postfix,is_kaitei,ymd,next_month_day1,next_month_day2,sha256,md5,size,lastfmodification,filecreation
0,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,05算定回数テーブル.csv,サンプル,osirase_04,05算定回数テーブル,,05算定回数テーブル,,,False,NaT,NaT,NaT,890542a2abd7d040ab78783bb838af1c456dcdd54e314e...,59b395aa7c467f838d818cd0bf2580cd,10.673828125kB,2023-11-22 10:07:26.207215,2023-11-22 10:07:26.207215
1,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,01補助マスターテーブル.csv,tensuhyo_02,tensuhyo_02_20120213,01補助マスターテーブル,,01補助マスターテーブル,,,False,NaT,NaT,NaT,38cfc25516491e252aef17f93ef4fa4de0ca64ee093485...,cf29e86e3c71970e1f4f88e1e74f5379,886.939453125kB,2023-11-22 10:07:26.234844,2023-11-22 10:07:26.234844
2,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,02包括テーブル.csv,tensuhyo_02,tensuhyo_02_20120213,02包括テーブル,,02包括テーブル,,,False,NaT,NaT,NaT,2b939f604846452eed5c891b4ad97d2b8d9633aa0d9f06...,70d068ccdf6093f9b24b963bb28062e1,6001.912109375kB,2023-11-22 10:07:26.317167,2023-11-22 10:07:26.317167
3,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,03-1背反テーブル1.csv,tensuhyo_02,tensuhyo_02_20120213,03-1背反テーブル1,1,03-1背反テーブル,1,,False,NaT,NaT,NaT,e9abb2b236605c230435dafe08e09bbf93941b402582a8...,9443610957dd09433753eb9230b2e1bf,578.05078125kB,2023-11-22 10:07:26.325180,2023-11-22 10:07:26.325180
4,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,03-2背反テーブル2.csv,tensuhyo_02,tensuhyo_02_20120213,03-2背反テーブル2,2,03-2背反テーブル,2,,False,NaT,NaT,NaT,92000a77113ec6044414d4f6a325a44463d850db653205...,fcb7bb8a12b2244dc42dffb0ed5ad644,392.81640625kB,2023-11-22 10:07:26.332179,2023-11-22 10:07:26.332179


医薬品マスターのみ

In [6]:
df_files_y =  df_files\
    .query("prefix == 'y' or prefix == 'y_ALL' ")\
    .drop_duplicates(subset=["filename", "sha256", "md5", "size"], keep='first', ) \
    .reset_index(drop=False)
    # 
    # .query("prefix == 'y_ALL' ")\
    

In [7]:
df_files_y.head()

,index,filepath,filename,parent,gparent,bearfile,timeinfo,prefix,ymdinfo,postfix,is_kaitei,ymd,next_month_day1,next_month_day2,sha256,md5,size,lastfmodification,filecreation
0,221,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,y20120326.csv,kihonmasta_04.files,h24,y20120326,20120326,y,20120326,,False,2012-03-26,2012-04-01,2012-04-02,9dbf111cce43d6b74ff7d0f1644a5bc73f259b9fb6058d...,9ab5c8b8e8e0a2a76cd644396566d77d,5.5087890625kB,2016-02-04 20:09:19,2023-11-22 09:57:51.723237
1,222,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,y20120402.csv,kihonmasta_04.files,h24,y20120402,20120402,y,20120402,,False,2012-04-02,2012-05-01,2012-05-02,59b4186b7a3be32756ba9b03de0698ab5807359d1a9057...,126eda053d3cc42071df996ebb1cd37c,7.2421875kB,2016-02-04 20:08:46,2023-11-22 09:57:51.552066
2,223,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,y20120417.csv,kihonmasta_04.files,h24,y20120417,20120417,y,20120417,,False,2012-04-17,2012-05-01,2012-05-02,092557400f8bb277174fb15da2e77af48e2c6e52fa6856...,5da05c67acd2fd4e6995a7c53656cdf1,5.0478515625kB,2016-02-04 20:08:36,2023-11-22 09:57:51.471866
3,224,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,y20120529.csv,kihonmasta_04.files,h24,y20120529,20120529,y,20120529,,False,2012-05-29,2012-06-01,2012-06-02,a81ff7ff59332cda09e8b044705506a2cddeae67d540d7...,0c15c7eedb0baedff2cd48d78f77c769,2.5341796875kB,2016-02-04 20:08:19,2023-11-22 09:57:51.394505
4,225,/mnt/d/kikin/www.ssk.or.jp/seikyushiharai/tens...,y20120531.csv,kihonmasta_04.files,h24,y20120531,20120531,y,20120531,,False,2012-05-31,2012-06-01,2012-06-02,7b2eb42b49c28587d95579efaf8a3a5eef51f4a83cb6b3...,4a172c529743cd503cc8cd713ddad66e,2.2783203125kB,2016-02-04 20:07:35,2023-11-22 09:57:51.306138


df_dt from step 2

In [8]:
# fdt_pk = "./datefile_20230426_100605_859194.pkl"
# fdt_pk = "./datefile_20230427_095353_581518.pkl"
fdt_pk = "./datefile_20231122_100722_065215.pkl"
dfdt = pd.read_pickle(fdt_pk)
dfdt.tail()

,dt,dt_str,yy_str,mm_str,dd_str,is_past
4986,2023-08-27,2023-08-27,2023,8,27,True
4987,2023-08-28,2023-08-28,2023,8,28,True
4988,2023-08-29,2023-08-29,2023,8,29,True
4989,2023-08-30,2023-08-30,2023,8,30,True
4990,2023-08-31,2023-08-31,2023,8,31,True


In [9]:
dt_start = dfdt["dt"].iat[0]
dt_start_str = dfdt["dt_str"].iat[0]
dt_start, dt_start_str

(Timestamp('2010-01-01 00:00:00'), '2010-01-01')

In [10]:
dt_end = dfdt["dt"].iat[-1]
dt_end_str = dfdt["dt_str"].iat[-1]
dt_end,dt_end_str

(Timestamp('2023-08-31 00:00:00'), '2023-08-31')

当該ファイル全部読み込み

In [11]:
%%time

l_df = []
for r in df_files_y.itertuples():
    f = r.filepath
    l_df.append(get_file_df(f))
# df_i.head()

CPU times: user 3.04 s, sys: 1.83 s, total: 4.88 s
Wall time: 6.38 s


In [12]:
def pick_dt(c_change, c_endsale, c_enduse):
    l_res = []
    for s_change, s_endsale, s_enduse in zip(c_change, c_endsale, c_enduse):
        if s_change != "":
            l_res.append(s_change)
        elif s_endsale != "" and s_enduse =="":
            l_res.append(s_endsale)
        elif s_enduse !="":
            l_res.append(s_enduse)
        else:
            l_res.append("")
    return l_res

In [13]:

df_whole = pd.concat(l_df)\
    .assign(c29 = lambda df:df.c29.fillna(""),
            c30 = lambda df:df.c30.fillna("").str.replace("99999999", ""),
            c33 = lambda df:df.c33.fillna("")
           )\
    .assign(ymdc = lambda df: pick_dt(df.c29, df.c30, df.c33 ))\
    .merge(df_files_y, on="filepath") \
    .assign(ymdinfo = lambda df: [r.ymdc if r.ymdc != "" else r.ymd for r in df.itertuples()]) \
    .assign(ymd = lambda df:pd.to_datetime(df.ymdinfo, errors="coerce"))\
    .sort_values(["c2", "ymd"])\
    .assign(next_month_day1 = lambda df:df.ymd + pd.offsets.MonthBegin(1)) \
    .assign(next_month_day2 = lambda df:df.next_month_day1 + datetime.timedelta(days =1))
df_whole.shape

(449143, 55)

In [14]:
# df_whole.query("ymd != ymd").head().T

In [15]:
df_whole.query("c33 ==''").head().T

,2802,4887,50315,4888,1012
c0,9,9,9,9,9
c1,Y,Y,Y,Y,Y
c2,610406002,610406008,610406013,610406015,610406020
c3,NaN,NaN,NaN,NaN,NaN
c4,NaN,NaN,NaN,NaN,NaN
c5,NaN,NaN,NaN,NaN,NaN
c6,NaN,NaN,NaN,NaN,NaN
c7,NaN,NaN,NaN,NaN,NaN
c8,NaN,NaN,NaN,NaN,NaN
c9,NaN,NaN,NaN,NaN,NaN


c4には薬剤名

In [16]:
df_whole\
    .query("c4 == c4").head().T

,6590,6591,6592,25163,52035
c0,5,5,5,0,9
c1,Y,Y,Y,Y,Y
c2,610406002,610406008,610406009,610406009,610406009
c3,14.0,9.0,13.0,13.0,13.0
c4,アストニール錠１０ １０ｍｇ,アゼピット錠１ｍｇ,アセメール錠１０ １０ｍｇ,アセメール錠１０ １０ｍｇ,アセメール錠１０ １０ｍｇ
c5,12.0,14.0,11.0,11.0,11.0
c6,ｱｽﾄﾆｰﾙｼﾞｮｳ10,ｱｾﾞﾋﾟｯﾄｼﾞｮｳ1MG,ｱｾﾒｰﾙｼﾞｮｳ10,ｱｾﾒｰﾙｼﾞｮｳ10,ｱｾﾒｰﾙｼﾞｮｳ10
c7,16.0,16.0,16.0,16.0,16.0
c8,1.0,1.0,1.0,1.0,1.0
c9,錠,錠,錠,錠,錠


c10の意味はレセプト電算処理システムマスターファイル使用説明書を参照されたい。

１：金額  
３：薬剤使用量省略（歯科に限る。）  
７：減点  

In [17]:
df_whole.query("c10 == 7 or c10 == 3") \
    .groupby(["c10", "c4"])\
    ["c2"]\
    .count()\
    .reset_index()\
    .set_index("c10")

,c4,c2
c10,,
3.0,アフタゾロン口腔用軟膏０．１％（１顎）,13
3.0,アフタゾロン口腔用軟膏０．１％（１／３顎）,13
3.0,アフタゾロン口腔用軟膏０．１％（２／３顎）,13
3.0,オルテクサー口腔用軟膏０．１％（１顎）,11
3.0,オルテクサー口腔用軟膏０．１％（１／３顎）,11
3.0,オルテクサー口腔用軟膏０．１％（２／３顎）,11
3.0,キシロカインポンプスプレー８％（ＯＡ１／３顎）,13
3.0,キシロカインポンプスプレー８％（ＯＡ１～２歯）,13
3.0,ケナログ口腔用軟膏０．１％（１顎）,3


In [18]:
df_whole\
    .groupby("c2")\
    ["c0"].count()\
    .reset_index() \
    .sort_values("c0", ascending = False)

,c2,c0
31460,662650050,24
31561,667140002,24
13884,620509101,23
10592,620007673,23
21662,622249401,22
...,...,...
1648,610443005,1
1647,610443004,1
31098,661310323,1
11967,620009118,1


In [19]:
df_whole\
    .groupby("c0")\
    ["c2"].count()\
    .reset_index() 
# \
#     .sort_values("c0", ascending = False)

,c0,c2
0,0,230751
1,3,13958
2,5,191044
3,9,13390


In [20]:
def c10c11fix(c10, c11):
    if c10 != c10 or c11 != c11:
        # c10 or c11がNAのならC11はそのまま
        return c11
    elif c11 ==0:
        return c11
    else:
        if c10 == 1:
            mul = 1
        elif c10 == 5:
            mul = 10
        elif c10 == 4:
            mul = 0.1
        else:
            #rint(c10, c11)
            mul=1
        return mul*c11

In [21]:
# s1 > s2　除外する。これは直近のマスターで情報があった項目で、翌月2日よりまえにdt_endが存在する場合である。

In [22]:
%%time
    # .assign(c11fill_ = lambda df:df.groupby("c2")["c11"].fillna(method="ffill"))\
    # .assign(c10fill_ = lambda df:df.groupby("c2")["c10"].fillna(method="ffill"))\

df_sub = df_whole\
    .loc[:, ["c0", "c2", "c4", "c9", "c10", "c11", "c16","c20","c23", "c24", "c27" ,"c31",
             "ymd", "next_month_day1", "next_month_day2", "filename"]].copy()\
    .query("c10 != 7 and c10 != 3") \
    .sort_values(["c2", "ymd"]) \
    .assign(c11fill_ = lambda df:df.groupby("c2")["c11"].ffill())\
    .assign(c10fill_ = lambda df:df.groupby("c2")["c10"].ffill())\
    .assign(c11fill = lambda df:[c10c11fix(r.c10fill_, r.c11fill_) for r in df.itertuples()])\
    .assign(c24fill = lambda df:[c10c11fix(r.c23, r.c24) for r in df.itertuples()])\
    .assign(s0 = lambda df:df.groupby("c2")["next_month_day2"].shift(1))\
    .assign(s1 = lambda df:df.next_month_day2)\
    .assign(s2 = lambda df:df.groupby("c2")["next_month_day2"].shift(-1))\
    .assign(dt_rank = lambda df:df.groupby("c2")["ymd"].rank(method='min'))\
    .assign(is_first_in_master = lambda df:np.isnan(df.s0))\
    .assign(is_last_in_master = lambda df:np.isnan(df.s2))\
    .assign(s0 = lambda df:df.s0.fillna(dt_start))\
    .assign(s2 = lambda df:df.s2.fillna(dt_end))\
    .query("s1 <= s2")
df_sub.head(10)

CPU times: user 5.3 s, sys: 124 ms, total: 5.42 s
Wall time: 5.42 s


,c0,c2,c4,c9,c10,c11,c16,c20,c23,c24,c27,c31,ymd,next_month_day1,next_month_day2,filename,c11fill_,c10fill_,c11fill,c24fill,s0,s1,s2,dt_rank,is_first_in_master,is_last_in_master
6590,5,610406002,アストニール錠１０ １０ｍｇ,錠,1.0,13.1,1.0,0.0,1.0,14.1,1.0,2123014F2031,2012-04-01,2012-05-01,2012-05-02,y20120305.csv,13.1,1.0,13.1,14.1,2010-01-01,2012-05-02,2012-12-02,1.0,True,False
1573,5,610406002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-11-30,2012-12-01,2012-12-02,y20121130.csv,13.1,1.0,13.1,NaN,2012-05-02,2012-12-02,2013-04-02,2.0,False,False
2802,9,610406002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-03-31,2013-04-01,2013-04-02,y20130329.csv,13.1,1.0,13.1,NaN,2012-12-02,2013-04-02,2023-08-31,3.0,False,True
6591,5,610406008,アゼピット錠１ｍｇ,錠,1.0,6.2,1.0,0.0,1.0,6.9,1.0,4490004F2010,2012-04-01,2012-05-01,2012-05-02,y20120305.csv,6.2,1.0,6.2,6.9,2010-01-01,2012-05-02,2013-10-02,1.0,True,False
3553,5,610406008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-09-30,2013-10-01,2013-10-02,y20130611.csv,6.2,1.0,6.2,NaN,2012-05-02,2013-10-02,2013-10-02,2.0,False,False
4887,9,610406008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-09-30,2013-10-01,2013-10-02,y20130930.csv,6.2,1.0,6.2,NaN,2013-10-02,2013-10-02,2023-08-31,2.0,False,True
6592,5,610406009,アセメール錠１０ １０ｍｇ,錠,1.0,13.1,1.0,0.0,1.0,14.1,1.0,2123014F2040,2012-04-01,2012-05-01,2012-05-02,y20120305.csv,13.1,1.0,13.1,14.1,2010-01-01,2012-05-02,2013-07-02,1.0,True,False
3674,5,610406009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-06-21,2013-07-01,2013-07-02,y20130621.csv,13.1,1.0,13.1,NaN,2012-05-02,2013-07-02,2013-07-02,2.0,False,False
25163,0,610406009,アセメール錠１０ １０ｍｇ,錠,1.0,13.1,1.0,0.0,1.0,14.1,1.0,2123014F2040,2013-06-21,2013-07-01,2013-07-02,y_ALL20140203.csv,13.1,1.0,13.1,14.1,2013-07-02,2013-07-02,2014-05-02,2.0,False,False
52035,9,610406009,アセメール錠１０ １０ｍｇ,錠,1.0,13.1,1.0,0.0,1.0,13.1,1.0,2123014F2040,2014-04-01,2014-05-01,2014-05-02,y20140305.csv,13.1,1.0,13.1,13.1,2013-07-02,2014-05-02,2023-08-31,4.0,False,True


In [23]:
#     .shape
# df_sub.query("c2==641140136").T

In [24]:
# df_whole.query("c2==641140136").T

In [25]:
# df_sub\
#     .query("s1 <= s2")\
# #     .shape
# df_sub.query("c2==641140136").T

In [26]:
df_whole.shape

(449143, 55)

In [27]:
df_sub.shape

(447125, 26)

In [28]:
df_sub.query("s1 > s2")

,c0,c2,c4,c9,c10,c11,c16,c20,c23,c24,c27,c31,ymd,next_month_day1,next_month_day2,filename,c11fill_,c10fill_,c11fill,c24fill,s0,s1,s2,dt_rank,is_first_in_master,is_last_in_master


In [29]:
# df_sub.query("c2 == 610406229")

## bsl: Before SaLe  
上市前。DB上、上市前に当該コードが存在するかは不明である。なので最も古い薬価を与える。
## bdm: Before Degital Master  
最古のデジタルマスターより前。デジタルマスターではそれ以前の値は不明である。なので最も古い薬価を与える。  
## ws: Within Sale  
セール期間中。最新の価格改定は翌月２日に反映される。  
## aes: After End of Sale
販売終了後。DB上、販売終了後に当該コードが存在するかは不明である。なので最も新しい薬価を与える。

In [30]:
def get_1st_tag (c0):
    if c0 == 3:
        return "bsl"
    elif c0 == 5:
        return "bdm"
    elif c0 == 0:
        return "bdm"
    elif c0 == 9:
        return "bdm"
    else:
        return "unknown"

In [31]:
df_sub\
    .query("c10 == c10")\
    .groupby("c10")\
    ["c2"].count()

c10
1.0    430271
4.0         4
5.0         8
Name: c2, dtype: int64

上記から、医薬品はほとんど価格で記載されている。

In [32]:
df_sub\
    .query("c10 == c10")\
    .query("c10 != 1")

,c0,c2,c4,c9,c10,c11,c16,c20,c23,c24,c27,c31,ymd,next_month_day1,next_month_day2,filename,c11fill_,c10fill_,c11fill,c24fill,s0,s1,s2,dt_rank,is_first_in_master,is_last_in_master
160891,3,629700201,ステミラック注,回分,5.0,1495775.50,0.0,0.0,0.0,0.00,4.0,4900401X1026,2019-02-26,2019-03-01,2019-03-02,y20190227.csv,1495775.50,5.0,1.495776e+07,0.000000e+00,2010-01-01,2019-03-02,2019-03-02,1.0,True,False
181483,0,629700201,ステミラック注,回分,5.0,1495775.50,0.0,0.0,0.0,0.00,4.0,4900401X1026,2019-02-26,2019-03-01,2019-03-02,y_ALL20190903.csv,1495775.50,5.0,1.495776e+07,0.000000e+00,2019-03-02,2019-03-02,2019-11-02,1.0,False,False
357077,5,629700201,ステミラック注,回分,5.0,1523475.00,0.0,0.0,5.0,1495775.50,4.0,4900401X1026,2019-10-01,2019-11-01,2019-11-02,y20190823.csv,1523475.00,5.0,1.523475e+07,1.495776e+07,2019-03-02,2019-11-02,2019-11-02,3.0,False,False
378666,0,629700201,ステミラック注,回分,5.0,1523475.00,0.0,0.0,5.0,1495775.50,4.0,4900401X1026,2019-10-01,2019-11-01,2019-11-02,y_ALL20200121.csv,1523475.00,5.0,1.523475e+07,1.495776e+07,2019-11-02,2019-11-02,2020-05-02,3.0,False,False
161743,3,629700401,キムリア点滴静注,患者当り,5.0,3349340.70,0.0,0.0,0.0,0.00,4.0,4900402X1020,2019-05-22,2019-06-01,2019-06-02,y20190521.csv,3349340.70,5.0,3.349341e+07,0.000000e+00,2010-01-01,2019-06-02,2019-06-02,1.0,True,False
181485,0,629700401,キムリア点滴静注,患者当り,5.0,3349340.70,0.0,0.0,0.0,0.00,4.0,4900402X1020,2019-05-22,2019-06-01,2019-06-02,y_ALL20190903.csv,3349340.70,5.0,3.349341e+07,0.000000e+00,2019-06-02,2019-06-02,2019-11-02,1.0,False,False
357078,5,629700401,キムリア点滴静注,患者当り,5.0,3411365.50,0.0,0.0,5.0,3349340.70,4.0,4900402X1020,2019-10-01,2019-11-01,2019-11-02,y20190823.csv,3411365.50,5.0,3.411366e+07,3.349341e+07,2019-06-02,2019-11-02,2019-11-02,3.0,False,False
378668,0,629700401,キムリア点滴静注,患者当り,5.0,3411365.50,0.0,0.0,5.0,3349340.70,4.0,4900402X1020,2019-10-01,2019-11-01,2019-11-02,y_ALL20200121.csv,3411365.50,5.0,3.411366e+07,3.349341e+07,2019-11-02,2019-11-02,2020-05-02,3.0,False,False
92925,5,667120013,NaN,NaN,4.0,2.55,NaN,NaN,NaN,NaN,NaN,NaN,2016-04-01,2016-05-01,2016-05-02,y20160318.csv,2.55,4.0,2.550000e-01,NaN,2014-05-02,2016-05-02,2016-05-02,5.0,False,False
138413,0,667120013,ベントナイト,ｇ,4.0,2.55,0.0,0.0,1.0,0.17,6.0,7129700X1016,2016-04-01,2016-05-01,2016-05-02,y_ALL20180313.csv,2.55,4.0,2.550000e-01,1.700000e-01,2016-05-02,2016-05-02,2018-05-02,5.0,False,False


キムリア、ステミラック注のc10=5については、以下の記載を参考にしている。  
https://www.ssk.or.jp/seikyushiharai/tensuhyo/kihonmasta/h30/kihonmasta_04.files/y_oshirase_20190225.pdf  
１ ： 金額  
３ ： 薬剤使用量省略（ 歯科に限る。）  
４ ： 除算金額（ 金額を１ ０ で除す。）  
５ ： 乗算金額（ 金額を１ ０ で乗ずる。）  
７ ： 減点  

c0の意味は  
０：前マスターの内容と同じであることを表す。  
１：抹消  
３：新規  
５：変更  
９：廃止  

新規。上市開始。

In [33]:
df_sub0 = df_sub.query("dt_rank == 1 and c0 ==3")\
    .rename(columns={
    "c11fill":"point",
    "s0":"t1",
    "s1":"t2"
}) \
    .loc[:,["c0", "c2","t1", "t2", "point", "ymd"]]\
    .assign(tag = "bsl",
            ind = 0)
df_sub0.head(5)

,c0,c2,t1,t2,point,ymd,tag,ind
47503,3,620033801,2010-01-01,2015-01-02,6.2,2014-12-12,bsl,0
3971,3,620034201,2010-01-01,2013-07-02,5.4,2013-06-21,bsl,0
47504,3,620034601,2010-01-01,2015-01-02,5.4,2014-12-12,bsl,0
5672,3,620034702,2010-01-01,2014-01-02,5.4,2013-12-13,bsl,0
183554,3,620034704,2010-01-01,2023-01-02,5.5,2022-12-09,bsl,0


初出だが新規ではない。

In [34]:
df_sub1 = df_sub\
    .query("dt_rank == 1 and c0 !=3")\
    .rename(columns={
    "c24fill":"point",
    "s0":"t1",
    "s1":"t2"
}) \
    .loc[:,["c0", "c2","t1", "t2", "point", "ymd"]]\
    .assign(tag = "bdm",
            ind = 0)
    # .drop_duplicates()
df_sub1.head(5)

,c0,c2,t1,t2,point,ymd,tag,ind
6590,5,610406002,2010-01-01,2012-05-02,14.1,2012-04-01,bdm,0
6591,5,610406008,2010-01-01,2012-05-02,6.9,2012-04-01,bdm,0
6592,5,610406009,2010-01-01,2012-05-02,14.1,2012-04-01,bdm,0
6593,5,610406013,2010-01-01,2012-05-02,6.9,2012-04-01,bdm,0
25164,0,610406013,2012-05-02,2012-05-02,6.9,2012-04-01,bdm,0


In [35]:
df_sub1.shape

(44293, 8)

終了ではない

In [36]:
df_sub2 = df_sub \
    .query("c0 != 9")\
    .rename(columns={
    "c11fill":"point",
    "s1":"t1",
    "s2":"t2"
}) \
    .loc[:,["c0","c2","t1", "t2", "point", "ymd"]] \
    .assign(tag = "ws",
            ind = 1)
df_sub2.head()

,c0,c2,t1,t2,point,ymd,tag,ind
6590,5,610406002,2012-05-02,2012-12-02,13.1,2012-04-01,ws,1
1573,5,610406002,2012-12-02,2013-04-02,13.1,2012-11-30,ws,1
6591,5,610406008,2012-05-02,2013-10-02,6.2,2012-04-01,ws,1
3553,5,610406008,2013-10-02,2013-10-02,6.2,2013-09-30,ws,1
6592,5,610406009,2012-05-02,2013-07-02,13.1,2012-04-01,ws,1


廃止

In [37]:
df_sub3 = df_sub \
    .query("c0 == 9")\
    .rename(columns={
    "c11fill":"point",
    "s1":"t1",
    "s2":"t2"
}) \
    .loc[:,["c0","c2","t1", "t2", "point", "ymd"]] \
    .assign(tag = "aes", ind = 2)
df_sub3.head()

,c0,c2,t1,t2,point,ymd,tag,ind
2802,9,610406002,2013-04-02,2023-08-31,13.1,2013-03-31,aes,2
4887,9,610406008,2013-10-02,2023-08-31,6.2,2013-09-30,aes,2
52035,9,610406009,2014-05-02,2023-08-31,13.1,2014-04-01,aes,2
50315,9,610406013,2015-10-02,2023-08-31,5.8,2015-09-30,aes,2
4888,9,610406015,2013-10-02,2023-08-31,11.1,2013-09-30,aes,2


つなげる

In [38]:
df_span = pd.concat([df_sub0, df_sub1, df_sub2, df_sub3])\
    .sort_values(["c2", "ymd", "ind"])\
    .query("t1 != t2")
df_span.head(50)

,c0,c2,t1,t2,point,ymd,tag,ind
6590,5,610406002,2010-01-01,2012-05-02,14.1,2012-04-01,bdm,0
6590,5,610406002,2012-05-02,2012-12-02,13.1,2012-04-01,ws,1
1573,5,610406002,2012-12-02,2013-04-02,13.1,2012-11-30,ws,1
2802,9,610406002,2013-04-02,2023-08-31,13.1,2013-03-31,aes,2
6591,5,610406008,2010-01-01,2012-05-02,6.9,2012-04-01,bdm,0
6591,5,610406008,2012-05-02,2013-10-02,6.2,2012-04-01,ws,1
4887,9,610406008,2013-10-02,2023-08-31,6.2,2013-09-30,aes,2
6592,5,610406009,2010-01-01,2012-05-02,14.1,2012-04-01,bdm,0
6592,5,610406009,2012-05-02,2013-07-02,13.1,2012-04-01,ws,1
25163,0,610406009,2013-07-02,2014-05-02,13.1,2013-06-21,ws,1


In [39]:
df_sub\
    .query("c2==629700401")

,c0,c2,c4,c9,c10,c11,c16,c20,c23,c24,c27,c31,ymd,next_month_day1,next_month_day2,filename,c11fill_,c10fill_,c11fill,c24fill,s0,s1,s2,dt_rank,is_first_in_master,is_last_in_master
161743,3,629700401,キムリア点滴静注,患者当り,5.0,3349340.7,0.0,0.0,0.0,0.0,4.0,4900402X1020,2019-05-22,2019-06-01,2019-06-02,y20190521.csv,3349340.7,5.0,33493407.0,0.0,2010-01-01,2019-06-02,2019-06-02,1.0,True,False
181485,0,629700401,キムリア点滴静注,患者当り,5.0,3349340.7,0.0,0.0,0.0,0.0,4.0,4900402X1020,2019-05-22,2019-06-01,2019-06-02,y_ALL20190903.csv,3349340.7,5.0,33493407.0,0.0,2019-06-02,2019-06-02,2019-11-02,1.0,False,False
357078,5,629700401,キムリア点滴静注,患者当り,5.0,3411365.5,0.0,0.0,5.0,3349340.7,4.0,4900402X1020,2019-10-01,2019-11-01,2019-11-02,y20190823.csv,3411365.5,5.0,34113655.0,33493407.0,2019-06-02,2019-11-02,2019-11-02,3.0,False,False
378668,0,629700401,キムリア点滴静注,患者当り,5.0,3411365.5,0.0,0.0,5.0,3349340.7,4.0,4900402X1020,2019-10-01,2019-11-01,2019-11-02,y_ALL20200121.csv,3411365.5,5.0,34113655.0,33493407.0,2019-11-02,2019-11-02,2020-05-02,3.0,False,False
400390,0,629700401,キムリア点滴静注,患者当り,1.0,34113655.0,0.0,0.0,1.0,34113655.0,4.0,4900402X1020,2020-04-01,2020-05-01,2020-05-02,y20200305.csv,34113655.0,1.0,34113655.0,34113655.0,2019-11-02,2020-05-02,2021-05-02,5.0,False,False
424261,0,629700401,キムリア点滴静注,患者当り,1.0,34113655.0,0.0,0.0,1.0,34113655.0,4.0,4900402X1020,2021-04-01,2021-05-01,2021-05-02,y_20210310.csv,34113655.0,1.0,34113655.0,34113655.0,2020-05-02,2021-05-02,2021-08-02,6.0,False,False
426667,5,629700401,NaN,NaN,NaN,32647761.0,NaN,NaN,NaN,NaN,NaN,NaN,2021-07-01,2021-08-01,2021-08-02,y_20210630.csv,32647761.0,1.0,32647761.0,NaN,2021-05-02,2021-08-02,2021-08-02,7.0,False,False
447534,0,629700401,キムリア点滴静注,患者当り,1.0,32647761.0,0.0,0.0,1.0,34113655.0,4.0,4900402X1020,2021-07-01,2021-08-01,2021-08-02,y_ALL20220121.csv,32647761.0,1.0,32647761.0,34113655.0,2021-08-02,2021-08-02,2022-05-02,7.0,False,False
206119,0,629700401,キムリア点滴静注,患者当り,1.0,32647761.0,0.0,0.0,1.0,32647761.0,4.0,4900402X1020,2022-04-01,2022-05-01,2022-05-02,y_20220304.csv,32647761.0,1.0,32647761.0,32647761.0,2021-08-02,2022-05-02,2022-05-02,9.0,False,False
227796,0,629700401,キムリア点滴静注,患者当り,1.0,32647761.0,0.0,0.0,1.0,32647761.0,4.0,4900402X1020,2022-04-01,2022-05-01,2022-05-02,y_20220311.csv,32647761.0,1.0,32647761.0,32647761.0,2022-05-02,2022-05-02,2022-05-02,9.0,False,False


In [40]:
# .query("c2==641140136")
df_span\
    .query("c2==629700401")

,c0,c2,t1,t2,point,ymd,tag,ind
161743,3,629700401,2010-01-01,2019-06-02,33493407.0,2019-05-22,bsl,0
181485,0,629700401,2019-06-02,2019-11-02,33493407.0,2019-05-22,ws,1
378668,0,629700401,2019-11-02,2020-05-02,34113655.0,2019-10-01,ws,1
400390,0,629700401,2020-05-02,2021-05-02,34113655.0,2020-04-01,ws,1
424261,0,629700401,2021-05-02,2021-08-02,34113655.0,2021-04-01,ws,1
447534,0,629700401,2021-08-02,2022-05-02,32647761.0,2021-07-01,ws,1
271150,0,629700401,2022-05-02,2023-05-02,32647761.0,2022-04-01,ws,1
336275,0,629700401,2023-05-02,2023-08-31,32647761.0,2023-04-01,ws,1


In [41]:
df_span\
    .query("c2==641140136")

,c0,c2,t1,t2,point,ymd,tag,ind
23345,5,641140136,2010-01-01,2012-05-02,95.0,2012-04-01,bdm,0
43658,0,641140136,2012-05-02,2014-05-02,92.0,2012-04-01,ws,1
91442,0,641140136,2014-05-02,2016-05-02,92.0,2014-04-01,ws,1
137112,0,641140136,2016-05-02,2018-05-02,92.0,2016-04-01,ws,1
181836,0,641140136,2018-05-02,2019-11-02,92.0,2018-04-01,ws,1
379075,0,641140136,2019-11-02,2020-05-02,94.0,2019-10-01,ws,1
400751,0,641140136,2020-05-02,2020-10-02,94.0,2020-04-01,ws,1
403029,9,641140136,2020-10-02,2023-08-31,94.0,2020-09-30,aes,2


In [42]:
dfdt.head()

,dt,dt_str,yy_str,mm_str,dd_str,is_past
0,2010-01-01,2010-01-01,2010,1,1,True
1,2010-01-02,2010-01-02,2010,1,2,True
2,2010-01-03,2010-01-03,2010,1,3,True
3,2010-01-04,2010-01-04,2010,1,4,True
4,2010-01-05,2010-01-05,2010,1,5,True


この処理が重い。全く美しくない。

In [43]:
%%time
df_meta = df_whole \
    .loc[:, ["c0", "c2", "c4", "c9", "c10",
             "c13", "c14","c15", "c16", "c31",
             "ymd", "next_month_day1", "next_month_day2", "filename"]].copy()\
    .query("c10 != 7 and c10 != 3") \
    .sort_values(["c2", "ymd","filename"]) \
    .assign(c4 = lambda df:df.groupby("c2")["c4"].fillna(method ="ffill").fillna(method ="bfill").fillna("NA"))\
    .assign(c9 = lambda df:df.groupby("c2")["c9"].fillna(method ="ffill").fillna("NA"))\
    .assign(c13 = lambda df:df.groupby("c2")["c13"].fillna(method ="ffill").fillna(999))\
    .assign(c14 = lambda df:df.groupby("c2")["c14"].fillna(method ="ffill").fillna(999))\
    .assign(c15 = lambda df:df.groupby("c2")["c15"].fillna(method ="ffill").fillna(999))\
    .assign(c16 = lambda df:df.groupby("c2")["c16"].fillna(method ="ffill").fillna(999))\
    .assign(c31 = lambda df:df.groupby("c2")["c31"].fillna(method ="ffill").fillna(999))\
    .assign(dt_rank_neg = lambda df:df.groupby("c2")["ymd"].rank(ascending = False,method='first'))\
    .query("dt_rank_neg ==1")    
# .query("c2 == 610406079")
    
df_meta.head(50)

<timed exec>:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
<timed exec>:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
<timed exec>:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
<timed exec>:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
<timed exec>:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
<timed exec>:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
<timed exec>:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future ve

CPU times: user 6min 23s, sys: 2.44 s, total: 6min 25s
Wall time: 6min 25s


,c0,c2,c4,c9,c10,c13,c14,c15,c16,c31,ymd,next_month_day1,next_month_day2,filename,dt_rank_neg
2802,9,610406002,アストニール錠１０ １０ｍｇ,錠,NaN,0.0,0.0,0.0,1.0,2123014F2031,2013-03-31,2013-04-01,2013-04-02,y20130329.csv,1.0
3553,5,610406008,アゼピット錠１ｍｇ,錠,NaN,0.0,0.0,0.0,1.0,4490004F2010,2013-09-30,2013-10-01,2013-10-02,y20130611.csv,1.0
52035,9,610406009,アセメール錠１０ １０ｍｇ,錠,1.0,0.0,0.0,0.0,1.0,2123014F2040,2014-04-01,2014-05-01,2014-05-02,y20140305.csv,1.0
49059,5,610406013,アドメッセン錠１ｍｇ,錠,NaN,0.0,0.0,0.0,1.0,4490004F2010,2015-09-30,2015-10-01,2015-10-02,y20150612.csv,1.0
4888,9,610406015,アナシロール錠１０ １０ｍｇ,錠,NaN,0.0,0.0,0.0,1.0,2123014F2058,2013-09-30,2013-10-01,2013-10-02,y20130930.csv,1.0
228,5,610406020,アルカルロールカプセル０．２５ ０．２５μｇ,カプセル,NaN,0.0,0.0,0.0,1.0,3112004M1015,2012-09-30,2012-10-01,2012-10-02,y20120613.csv,1.0
229,5,610406021,アルカルロールカプセル０．５ ０．５μｇ,カプセル,NaN,0.0,0.0,0.0,1.0,3112004M2011,2012-09-30,2012-10-01,2012-10-02,y20120613.csv,1.0
337888,9,610406047,ウテロン錠５ｍｇ,錠,1.0,0.0,0.0,0.0,1.0,2590004F1010,2019-10-01,2019-11-01,2019-11-02,y20190823.csv,1.0
45719,5,610406050,エスメラルダ錠４００ ４００ｍｇ,錠,NaN,0.0,0.0,0.0,1.0,2325001F2013,2014-09-30,2014-10-01,2014-10-02,y20140613.csv,1.0
96919,9,610406053,エマベリンＬカプセル５ｍｇ,カプセル,1.0,0.0,0.0,0.0,1.0,2171014N4028,2016-04-01,2016-05-01,2016-05-02,y20160304.csv,1.0


In [44]:
# df_whole.query("c2 ==630010001").tail().T

In [45]:
df_meta.query("c2 == 610406002")

,c0,c2,c4,c9,c10,c13,c14,c15,c16,c31,ymd,next_month_day1,next_month_day2,filename,dt_rank_neg
2802,9,610406002,アストニール錠１０ １０ｍｇ,錠,NaN,0.0,0.0,0.0,1.0,2123014F2031,2013-03-31,2013-04-01,2013-04-02,y20130329.csv,1.0


In [46]:
[c2 for c2 in list(df_sub["c2"].unique()) if c2 not in df_meta["c2"].to_list()]

[]

In [47]:
d_output = "../output_iy/"
outputtag = "iycode_"
# postfix = ".json.gz"
# postfix = ".json.bz2"
postfix = ".json"
pf = d_output + outputtag + "{}" + "_" + ts + postfix
pf

'../output_iy/iycode_{}_20231122_150053_929635.json'

In [48]:
import json
# import gzip
import bz2
import joblib
# with gzip.open(write_file, 'wt', encoding="ascii") as zipfile:
#        json.dump(data, zipfile)
        
def d2j(d, f_json):
    with open(f_json,  mode='wt', encoding='utf-8') as file:
        json.dump(d, file, ensure_ascii=False, indent=2)

def d2jz(d, f_json):
    # with gzip.open(f_json,  mode='wt', encoding='utf-8') as zipfile:
    with bz2.open(f_json,  mode='wt', encoding='utf-8') as zipfile:
        json.dump(d, zipfile, ensure_ascii=False, indent=2)


いまさらだが書き出しは並列でやったほうが早い。  
当初ベタなループであったが、並列処理に修正。  
コアもりもりの計算機ならすぐ終わるはず。

In [49]:
def c2j(c2):
    df_iter = dfdt.copy()\
        .loc[:,["dt","dt_str"]]\
        .assign(c2 = c2)\
        .merge(df_span, on= "c2")\
        .query("t1 <= dt and dt < t2")\
        .assign(point = lambda df:df.point.fillna(0))
    # point情報が無いものは0で穴埋めしている。
    
    d_meta_sub = df_meta\
    .query("c2 == @c2").to_dict('records')[0]

    f_out = pf.format(c2)
    # print(f_out)
    d_i = {
        "drugcode":int(c2),
        "name":d_meta_sub["c4"],
        "nplcode":d_meta_sub["c31"],
        "unit":d_meta_sub["c9"],
        "cat_for_mental":d_meta_sub["c13"],
        "cat_neuro_break": d_meta_sub["c14"],
        "cat_bio":d_meta_sub["c15"],
        "cat_generic":d_meta_sub["c16"],
        "dt_start": dt_start_str,
        "dt_end":dt_end_str,
        "dt":dict()
    }
    
    for r in df_iter.itertuples():
        d_i["dt"][r.dt_str] = {"tag":r.tag,
                              "point":r.point}
    
    # print(d_i)
    # d2jz(d=d_i, f_json=f_out)
    d2j(d=d_i, f_json=f_out)
    return f_out

c2list = list(df_sub["c2"].unique())
len(c2list)

31587

In [56]:
%%time
result = joblib.Parallel(n_jobs=-5,verbose=2)(joblib.delayed(c2j)(i) for i in c2list)

[Parallel(n_jobs=-5)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-5)]: Done  33 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-5)]: Done 154 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-5)]: Done 357 tasks      | elapsed:   42.7s
[Parallel(n_jobs=-5)]: Done 640 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-5)]: Done 1005 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-5)]: Done 1450 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-5)]: Done 1977 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-5)]: Done 2584 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-5)]: Done 3273 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-5)]: Done 4042 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-5)]: Done 4893 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-5)]: Done 5824 tasks      | elapsed: 15.4min
[Parallel(n_jobs=-5)]: Done 6837 tasks      | elapsed: 17.5min
[Parallel(n_jobs=-5)]: Done 7930 tasks      | elapsed: 19.7min
[Parallel(n_jobs=-5)]: Done 9105 tasks      | 

CPU times: user 25min 53s, sys: 1min 43s, total: 27min 37s
Wall time: 45min 31s


In [57]:
f_meta_x = "../appendix/dfmeta_iy_" + ts + ".xlsx"
f_meta_pk = "../appendix/dfmeta_iy_" + ts + ".pkl"

df_meta.to_excel(f_meta_x)
df_meta.to_pickle(f_meta_pk)
f_meta_x, f_meta_pk

('../appendix/dfmeta_iy_20231122_150053_929635.xlsx',
 '../appendix/dfmeta_iy_20231122_150053_929635.pkl')

In [58]:
f_span_x = "../appendix/dfspan_iy_" + ts + ".xlsx"
f_span_pk = "../appendix/dfspan_iy_" + ts + ".pkl"
df_span.to_excel(f_span_x)
df_span.to_pickle(f_span_pk)
f_span_x, f_span_pk

('../appendix/dfspan_iy_20231122_150053_929635.xlsx',
 '../appendix/dfspan_iy_20231122_150053_929635.pkl')

In [53]:
 # 630010006

In [54]:
df_whole\
    .query("c2 == 630010006").head().T

,48321,91013,115683,136733,157783
c0,3,0,0,0,5
c1,Y,Y,Y,Y,Y
c2,630010006,630010006,630010006,630010006,630010006
c3,31.0,31.0,31.0,31.0,31.0
c4,薬剤料逓減（６０／１００）（紹介率が低い大病院３０日以上投薬）,薬剤料逓減（６０／１００）（紹介率が低い大病院３０日以上投薬）,薬剤料逓減（６０／１００）（紹介率が低い大病院３０日以上投薬）,薬剤料逓減（６０／１００）（紹介率が低い大病院３０日以上投薬）,薬剤料逓減（４０／１００）（紹介率が低い大病院３０日以上投薬）
c5,13.0,13.0,13.0,13.0,13.0
c6,ﾔｸｻﾞｲﾘﾖｳﾃｲｹﾞﾝ,ﾔｸｻﾞｲﾘﾖｳﾃｲｹﾞﾝ,ﾔｸｻﾞｲﾘﾖｳﾃｲｹﾞﾝ,ﾔｸｻﾞｲﾘﾖｳﾃｲｹﾞﾝ,ﾔｸｻﾞｲﾘﾖｳﾃｲｹﾞﾝ
c7,0.0,0.0,0.0,0.0,0.0
c8,0.0,0.0,0.0,0.0,0.0
c9,NaN,NaN,NaN,NaN,NaN


In [55]:
df_meta\
    .query("c2 == 630010006")

,c0,c2,c4,c9,c10,c13,c14,c15,c16,c31,ymd,next_month_day1,next_month_day2,filename,dt_rank_neg
159481,5,630010006,薬評（内用薬）,NA,NaN,999.0,999.0,999.0,999.0,999,2018-04-01,2018-05-01,2018-05-02,y20180313.csv,1.0
